In [1]:
import os

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

!pip install -q findspark

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,398 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,434 kB]
Get:12 http://security.ubuntu.

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-10-26 15:46:59--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.15MB/s    in 0.2s    

2021-10-26 15:46:59 (5.15 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
 # Start Spark session
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, DateType
from pyspark import SparkFiles
from pyspark.sql.functions import col


# Read in data from S3 Bukets
spark = SparkSession.builder.appName("AmazonReviews_Toys").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()
url= "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Toys_v1_00.tsv.gz'), sep='\t', header=True, inferSchema=True)
df.show(n=5,truncate=False)

+-----------+-----------+--------------+----------+--------------+----------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+----------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|marketplace|customer_id|review_id     |product_id|product_parent|product_title                                                         |product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline                                           |review_body                                                                                                                                                                  

In [5]:
# Current schema type
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [6]:
# Count number of rows in dataset
df.count()

4864249

In [11]:
# Create review_id_table and verify schema
toys_review_id_table = df.select(df.review_id, df.customer_id, df.product_id, df.product_parent, df.review_date.astype(dataType=DateType()))
toys_review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [12]:
# Create products table and verify schema
toys_products = df.select(df.product_id, df.product_title)
toys_products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [10]:
# Create customers table and verify schema
toys_customers = df.groupBy('customer_id').count().withColumn('count',col('count').cast(IntegerType()))
toys_customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- count: integer (nullable = false)



In [9]:
# Create vine_table and verify schema
toys_vine_table = df.select(df.review_id, df.star_rating, df.helpful_votes, df.total_votes, df.vine)
toys_vine_table.printSchema()


root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://mypostgresdb.cryub0wqjex7.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root",
          "password": "g1EC1QdWl",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write dataframes to RDS
toys_review_id_table.write.jdbc(url=jdbc_url,
                         table ='toys_review_id_table', 
                         mode=mode, properties = config)

Py4JJavaError: ignored

In [ ]:
toys_products.write.jdbc(url=jdbc_url,
                         table ='toys_products', 
                         mode=mode, properties = config)

In [ ]:
toys_customers.write.jdbc(url=jdbc_url,
                         table ='toys_customers', 
                         mode=mode, properties = config)

In [ ]:
toys_vine_table.write.jdbc(url=jdbc_url,
                         table ='toys_vine_table', 
                         mode=mode, properties = config)